## Seoul & Gyeunggi realestate
  - Source url : https://price.joinsland.joins.com/theme/index_theme.asp?sisaegbn=T05

In [275]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from fake_useragent import UserAgent
import re
import time
from selenium.webdriver.firefox.options import Options

## User setting

In [276]:
browser_local_url = r'C:/Program Files/Mozilla Firefox/firefox.exe'
selenium_driver_local_url = "C:/Users/User/Documents/_SM folder/realestate/crawling_driver/geckodriver-v0.24.0-win64/geckodriver.exe"
region_set = 'gyeonggi' # option: seoul, gyeonggi

In [277]:
capabilities = webdriver.DesiredCapabilities().FIREFOX
capabilities["marionette"] = True
binary = FirefoxBinary(browser_local_url)
driver = webdriver.Firefox(firefox_binary=binary, 
                           capabilities=capabilities, 
                           executable_path=selenium_driver_local_url)

In [278]:
seould_xpath = '//*[@id="container"]/div/div/div[2]/div[1]/form[2]/select[1]/option[1]'
gg_xpath = '//*[@id="container"]/div/div/div[2]/div[1]/form[2]/select[1]/option[2]'

In [279]:
if region_set == 'seoul':
    region_xpath = seould_xpath
else:
    region_xpath = gg_xpath

print("You selected the region as", region_set)

You selected the region as gyeonggi


In [280]:
driver.get('https://price.joinsland.joins.com/theme/index_theme.asp?sisaegbn=T05')
driver.find_element_by_xpath(region_xpath).click()
time.sleep(3)
param_area_code = {'selected' : 'selected', 'option':'value'}
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
temp = soup.findAll('option')
region_code_df = pd.DataFrame({'region':[i.text for i in temp[17:]], 'code':[i.attrs['value'] for i in temp[17:]]})
region_code_df.to_csv('./region_code.csv')

In [281]:
region_code_df

,region,code
0,가평군,4182000000
1,고양시 덕양구,4128100000
2,고양시 일산동구,4128500000
3,고양시 일산서구,4128700000
4,과천시,4129000000
5,광명시,4121000000
6,광주시,4161000000
7,구리시,4131000000
8,군포시,4141000000
9,김포시,4157000000


In [282]:
param_lastpagenum = {'class':'paginate'}

In [283]:
main_url = 'https://price.joinsland.joins.com/theme/index_theme.asp?CSRFToken=HRnvjxPMunoWtnlqjimfZLIcUQvQiVkHZflhkaAy&sisaegbn=T05&areaCode='
page_url = '&searchgbn=&page='

In [284]:
region_code_df['url'] = main_url + region_code_df['code'] + page_url 

In [285]:
region_code_df.loc[:, 'url'][0]

'https://price.joinsland.joins.com/theme/index_theme.asp?CSRFToken=HRnvjxPMunoWtnlqjimfZLIcUQvQiVkHZflhkaAy&sisaegbn=T05&areaCode=4182000000&searchgbn=&page='

In [286]:
collecting_df = pd.DataFrame(columns=['region','ranking', 'address', 'danzi', 'are_num', 'year', 'household', 'per', 'buy_price', 'buy_num', 'rent_price', 'rent_num'])

In [287]:
collecting_df

,region,ranking,address,danzi,are_num,year,household,per,buy_price,buy_num,rent_price,rent_num


In [288]:
collecting_df_temp_initial = collecting_df.copy()
collecting_df_temp = collecting_df.copy()

In [289]:
ranking_param = {'class':'num_ranking'}
address_param = {'class':'txt_addres2'}
danzi_param = {'class':'txt_danzi'}
numarea_param = {'class':'num_area'}
num_year_param = {'class':'num_year'}
num_household_param = {'class':'num_household'}
num_per_param = {'class':'num_per'}

In [290]:
def parsing(parsed_list):
    return [i.text for i in parsed_list]

In [291]:
row = 0
for r, url in zip(region_code_df['region'], region_code_df['url']):
    driver.get(url)
    driver.implicitly_wait(5)
    try:
        driver.find_element_by_css_selector('#container > div > div > div.theme_effect > div.paginate > a.next').click()
        driver.implicitly_wait(5)
        driver.find_element_by_css_selector('#container > div > div > div.theme_effect > div.paginate > a.next').click()
        driver.implicitly_wait(5)
        driver.find_element_by_css_selector('#container > div > div > div.theme_effect > div.paginate > a.next').click()
        driver.implicitly_wait(5)
    except:
        pass
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        if soup.find('div', param_lastpagenum).select('a')[-1].text == '이전':
            last_page_num = int(soup.find('div', param_lastpagenum).select_one('strong').text)
        else:
            try:
                last_page_num = int(soup.find('div', param_lastpagenum).select('a')[-1].text)
            except:
                continue
    except:
        continue
        
    print(r)
    print('last_page_num')
    print(last_page_num)
    # generate the variables and loop it
    print('loop start')
    for i, p in enumerate(range(1, last_page_num+1)):
        try:
            collecting_df_temp = collecting_df_temp_initial.copy()
            driver.get(url+str(p))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            #soup.select('td:nth-of-type(8)') ~ soup.select('td:nth-of-type(11)')
            collecting_df_temp['ranking'] = parsing(soup.findAll('td', ranking_param))
            collecting_df_temp['address'] = parsing(soup.findAll('td', address_param))
            collecting_df_temp['danzi'] = parsing(soup.findAll('td', danzi_param))
            collecting_df_temp['are_num'] = parsing(soup.findAll('td', numarea_param))
            collecting_df_temp['year'] = parsing(soup.findAll('td', num_year_param))
            collecting_df_temp['household'] = parsing(soup.findAll('td', num_household_param))
            collecting_df_temp['per'] = parsing(soup.findAll('td', num_per_param))
            collecting_df_temp['buy_price'] = parsing(soup.select('td:nth-of-type(8)'))
            collecting_df_temp['buy_num'] = parsing(soup.select('td:nth-of-type(9)'))
            collecting_df_temp['rent_price'] = parsing(soup.select('td:nth-of-type(10)'))
            collecting_df_temp['rent_num'] = parsing(soup.select('td:nth-of-type(11)'))
            collecting_df_temp['region'] = [r]*collecting_df_temp.shape[0]
            #collecting_df row bind with collecting_df1~ ;;;
            collecting_df = pd.concat([collecting_df,collecting_df_temp], axis = 0)
            collecting_df.reset_index(drop=True, inplace = True)
        except:
            pass        
    print('loop end')
    collecting_df.to_csv('./output_df/' + region_set + '_collecting_df.csv')
            

가평군
last_page_num
2
loop start
loop end
고양시 덕양구
last_page_num
11
loop start
loop end
고양시 일산동구
last_page_num
9
loop start
loop end
고양시 일산서구
last_page_num
11
loop start
loop end
광명시
last_page_num
7
loop start
loop end
광주시
last_page_num
9
loop start
loop end
구리시
last_page_num
6
loop start
loop end
군포시
last_page_num
8
loop start
loop end
김포시
last_page_num
12
loop start
loop end
남양주시
last_page_num
21
loop start
loop end
동두천시
last_page_num
3
loop start
loop end
부천시
last_page_num
24
loop start
loop end
성남시 분당구
last_page_num
14
loop start
loop end
성남시 수정구
last_page_num
4
loop start
loop end
성남시 중원구
last_page_num
4
loop start
loop end
수원시 권선구
last_page_num
10
loop start
loop end
수원시 영통구
last_page_num
9
loop start
loop end
수원시 장안구
last_page_num
7
loop start
loop end
수원시 팔달구
last_page_num
4
loop start
loop end
시흥시
last_page_num
14
loop start
loop end
안산시 단원구
last_page_num
7
loop start
loop end
안산시 상록구
last_page_num
5
loop start
loop end
안성시
last_page_num
4
loop start
loop end
안양시 동안구
last_page_nu

## quick EDA

In [293]:
collecting_df

,region,ranking,address,danzi,are_num,year,household,per,buy_price,buy_num,rent_price,rent_num
0,가평군,1,청평면,청평로얄,105,2007,54,86.67%,"14,000 ~ 16,000",0,"12,000 ~ 14,000",0
1,가평군,2,청평면,청평로얄,110,2007,54,86.67%,"14,000 ~ 16,000",0,"12,000 ~ 14,000",0
2,가평군,3,설악면,북한강코아루,114A,2014,237,79.25%,"24,000 ~ 28,500",0,"20,000 ~ 22,000",0
3,가평군,4,설악면,북한강코아루,115B,2014,237,79.25%,"24,000 ~ 28,500",0,"20,000 ~ 22,000",0
4,가평군,5,설악면,북한강코아루,118C,2014,237,79.25%,"24,000 ~ 28,500",0,"20,000 ~ 22,000",0
...,...,...,...,...,...,...,...,...,...,...,...,...
16144,화성시,723,청계동,신안인스빌리베라1차,111A,2015,913,51.59%,"60,500 ~ 65,000",1,"30,000 ~ 34,500",0
16145,화성시,724,청계동,신안인스빌리베라1차,112B,2015,913,51.59%,"60,500 ~ 65,000",0,"30,000 ~ 34,500",0
16146,화성시,725,청계동,신안인스빌리베라2차,98,2016,644,51.67%,"56,500 ~ 62,500",1,"30,500 ~ 32,000",0
16147,화성시,726,목동,힐스테이트동탄,88,2019,"1,479",51.43%,"49,500 ~ 55,000",0,"25,500 ~ 28,000",0


In [294]:
collecting_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16149 entries, 0 to 16148
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   region      16149 non-null  object
 1   ranking     16149 non-null  object
 2   address     16149 non-null  object
 3   danzi       16149 non-null  object
 4   are_num     16149 non-null  object
 5   year        16149 non-null  object
 6   household   16149 non-null  object
 7   per         16149 non-null  object
 8   buy_price   16149 non-null  object
 9   buy_num     16149 non-null  object
 10  rent_price  16149 non-null  object
 11  rent_num    16149 non-null  object
dtypes: object(12)
memory usage: 1.5+ MB


In [297]:
collecting_df.year.unique()

array(['2007', '2014', '2018', '2006', '2012', '1992', '2019', '1998',
       '1993', '2008', '1996', '2001', '1997', '1994', '2003', '2009',
       '2004', '1995', '2005', '2010', '2000', '1999', '2017', '1991',
       '2015', '2013', '2011', '1987', '2002', '1988', '1986', '2016',
       '1989', '1990', '1985', '2020', '1984', '1979', '1983', '1982',
       '1978', '1980', '-'], dtype=object)

In [298]:
collecting_df['ranking'] = [int(i) for i in collecting_df['ranking'] ]
collecting_df['year'] = [1980 if i == '-' else int(i) for i in collecting_df['year'] ]
collecting_df['buy_num'] = [int(i) for i in collecting_df['buy_num'] ]
collecting_df['rent_num'] = [int(i) for i in collecting_df['rent_num'] ]

In [299]:
collecting_df['household'] = [int(i.replace(',','')) for i in collecting_df['household']]
collecting_df['per'] = [float(i.replace('%','')) for i in collecting_df['per']]

In [300]:
collecting_df['buy_price_min'] =[int(i.split(' ~ ')[0].replace(',','')) for i in collecting_df['buy_price']]
collecting_df['buy_price_max'] =[int(i.split(' ~ ')[1].replace(',','')) for i in collecting_df['buy_price']]

collecting_df['rent_price_min'] =[int(i.split(' ~ ')[0].replace(',','')) for i in collecting_df['rent_price']]
collecting_df['rent_price_max'] =[int(i.split(' ~ ')[1].replace(',','')) for i in collecting_df['rent_price']]

In [301]:
def parsing_area_num(area):
    try:
        return int(area[0:re.search(r'[가-힣a-zA-Z(-]', area).start()])
    except:
        return int(area)

In [302]:
collecting_df['are_num_int'] = collecting_df['are_num'].apply(lambda x:parsing_area_num(x))

In [303]:
collecting_df

,region,ranking,address,danzi,are_num,year,household,per,buy_price,buy_num,rent_price,rent_num,buy_price_min,buy_price_max,rent_price_min,rent_price_max,are_num_int
0,가평군,1,청평면,청평로얄,105,2007,54,86.67,"14,000 ~ 16,000",0,"12,000 ~ 14,000",0,14000,16000,12000,14000,105
1,가평군,2,청평면,청평로얄,110,2007,54,86.67,"14,000 ~ 16,000",0,"12,000 ~ 14,000",0,14000,16000,12000,14000,110
2,가평군,3,설악면,북한강코아루,114A,2014,237,79.25,"24,000 ~ 28,500",0,"20,000 ~ 22,000",0,24000,28500,20000,22000,114
3,가평군,4,설악면,북한강코아루,115B,2014,237,79.25,"24,000 ~ 28,500",0,"20,000 ~ 22,000",0,24000,28500,20000,22000,115
4,가평군,5,설악면,북한강코아루,118C,2014,237,79.25,"24,000 ~ 28,500",0,"20,000 ~ 22,000",0,24000,28500,20000,22000,118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16144,화성시,723,청계동,신안인스빌리베라1차,111A,2015,913,51.59,"60,500 ~ 65,000",1,"30,000 ~ 34,500",0,60500,65000,30000,34500,111
16145,화성시,724,청계동,신안인스빌리베라1차,112B,2015,913,51.59,"60,500 ~ 65,000",0,"30,000 ~ 34,500",0,60500,65000,30000,34500,112
16146,화성시,725,청계동,신안인스빌리베라2차,98,2016,644,51.67,"56,500 ~ 62,500",1,"30,500 ~ 32,000",0,56500,62500,30500,32000,98
16147,화성시,726,목동,힐스테이트동탄,88,2019,1479,51.43,"49,500 ~ 55,000",0,"25,500 ~ 28,000",0,49500,55000,25500,28000,88


In [304]:
collecting_df.columns

Index(['region', 'ranking', 'address', 'danzi', 'are_num', 'year', 'household',
       'per', 'buy_price', 'buy_num', 'rent_price', 'rent_num',
       'buy_price_min', 'buy_price_max', 'rent_price_min', 'rent_price_max',
       'are_num_int'],
      dtype='object')

In [305]:
preprocessed_df = collecting_df[['region', 'ranking', 'address', 'danzi', \
                                'are_num_int', 'year', 'household','per', \
                                'buy_num', 'buy_price_min', 'buy_price_max', \
                                 'rent_num', 'rent_price_min', 'rent_price_max']]

In [306]:
preprocessed_df.to_csv('preprocessed_final.csv', index = False)

In [307]:
preprocessed_df[(preprocessed_df['are_num_int'] > 75) & (preprocessed_df['are_num_int'] < 110) &\
               (preprocessed_df['per'] > 60) & (preprocessed_df['buy_price_min'] < 60000) &\
               (preprocessed_df['household'] > 700)]

,region,ranking,address,danzi,are_num_int,year,household,per,buy_num,buy_price_min,buy_price_max,rent_num,rent_price_min,rent_price_max
65,고양시 덕양구,12,행신동,햇빛18-1단지뜨란채,87,1998,1132,88.33,1,28000,32000,0,26000,27000
71,고양시 덕양구,18,고양동,푸른3단지동익미라벨,108,2004,958,87.78,0,21400,23500,0,19000,20500
72,고양시 덕양구,19,고양동,푸른3단지동익미라벨,108,2004,958,87.78,0,21400,23500,0,19000,20500
82,고양시 덕양구,29,고양동,푸른3단지동익미라벨,87,2004,958,86.25,0,19250,20500,0,16750,17750
87,고양시 덕양구,34,행신동,햇빛20단지주공,83,1997,1713,86.00,0,24000,26000,0,21000,22000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,화성시,464,석우동,동탄예당마을우미린제일풍경채,104,2008,1316,61.46,1,42000,50000,0,29000,30000
15886,화성시,465,석우동,동탄예당마을우미린제일풍경채,103,2008,1316,61.46,0,42000,50000,0,29000,30000
15901,화성시,480,반송동,동탄시범다은마을월드메르디앙반도유보라,81,2007,1473,60.42,0,46000,50000,0,28000,30000
15913,화성시,492,반월동,SK뷰파크,84,2015,1967,60.42,0,45000,50000,0,28000,30000


In [308]:
preprocessed_df[(preprocessed_df['are_num_int'] > 75) & (preprocessed_df['are_num_int'] < 110) &\
               (preprocessed_df['per'] > 60) & (preprocessed_df['buy_price_min'] < 60000) &\
               (preprocessed_df['household'] > 700)].groupby('address').count()[['region']]

,region
address,
가남읍,2
가능동,8
가사동,5
가좌동,6
갈곶동,4
...,...
호평동,8
화도읍,6
화서동,2


In [309]:
filtered_df = preprocessed_df[(preprocessed_df['are_num_int'] > 75) & (preprocessed_df['are_num_int'] < 110) &\
               (preprocessed_df['per'] > 60) & (preprocessed_df['buy_price_min'] < 70000) &\
               (preprocessed_df['household'] > 700)]

In [310]:
filtered_df.sort_values(by = 'buy_price_min', ascending = False)

,region,ranking,address,danzi,are_num_int,year,household,per,buy_num,buy_price_min,buy_price_max,rent_num,rent_price_min,rent_price_max
10029,안양시 동안구,394,호계동,평촌더샵아이파크,82,2019,1138,61.54,6,69250,73500,5,41000,46500
10030,안양시 동안구,395,호계동,평촌더샵아이파크,82,2019,1138,61.54,0,69250,73500,0,41000,46500
11712,용인시 기흥구,501,보정동,죽현마을동원로얄듀크,109,2006,706,65.99,0,69000,79250,0,46500,50000
15251,하남시,137,망월동,미사강변파밀리에,80,2014,976,62.68,1,68500,73500,0,42500,46500
15252,하남시,138,망월동,미사강변파밀리에,80,2014,976,62.68,0,68500,73500,0,42500,46500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,안성시,14,공도읍,산수화,81,2003,751,82.89,5,8750,10250,1,7250,8250
9480,안성시,13,공도읍,산수화,82,2003,751,82.89,4,8750,10250,0,7250,8250
9479,안성시,12,공도읍,산수화,81,2003,751,82.89,0,8750,9950,1,7250,8250
13660,이천시,92,부발읍,"이화1,2차",81,1996,927,84.85,0,7750,8750,0,6500,7500
